In [1]:
import os
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

# Setup Python path to include the project root
project_dir = Path.cwd().parent
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import the agent and state management class
from ProductDesigner.agents.safe_product_interviewer import SafeProductInterviewerAgent, logging
from ProductDesigner.graph_state import GraphState

# Imports neccessary to run the cells below
from pydantic import BaseModel
from typing import List
from langchain.output_parsers import PydanticOutputParser
from openai import AsyncOpenAI
import asyncio
import os

print('Setup complete. Necessary packages and paths are loaded.')

Setup complete. Necessary packages and paths are loaded.


# 1. Agent Initialization

In [2]:
# Define the initial product idea and create a mock state
initial_idea = "An AI-powered mutual fund recommendation system for indian MFs - fetching data from public or 3rd part API, produce vizualizations, track investments, suggest switches, analyze performanc eo fmultuaal funds, determine best MFs for a given risk type, optimal allocation"
mock_state = {"initial_idea": initial_idea, "qna_history": {}}

# Initialize the interviewer agent
interviewer = SafeProductInterviewerAgent()
print("Agent initialized successfully.")

questions = interviewer._get_planning_questions()
questions

2025-08-05 10:29:41,237 - INFO - LLMService initialized with model: gpt-4o
2025-08-05 10:29:41,278 - INFO - Web search is enabled.


Agent initialized successfully.


['What is the primary purpose of this application or system?',
 'Who are the main users or target audience of this application?',
 'What are the 3-5 core features needed for the MVP (Minimum Viable Product)?',
 'What specific problem does this solution solve for your users?',
 'Are there any existing solutions to this problem? How is yours different?',
 'What technologies or tech stack do you have in mind? Or would you like recommendations?',
 'What are the main user flows or journeys through the application?',
 'Are there any specific UI/UX requirements or preferences?',
 'What are your plans for data storage and management?',
 'Are there any specific security requirements or concerns?',
 'What is your timeline for development of the MVP?',
 'Are there any third-party integrations needed?',
 'What metrics would define success for this product?']

## Agent 1: Get the Answers of the Questions I have prepared previously

In [ ]:
class QA(BaseModel):
    question: str
    answer: str

class QALIST(BaseModel):
    questions: List[QA]

    def str(self):
        return '\n'.join([f"Question: {qa.question}\nAnswer: {qa.answer}" for qa in self.questions])

parser = PydanticOutputParser(pydantic_object=QALIST)
format_instructions = parser.get_format_instructions()

questions = interviewer._get_planning_questions()
str_questions = '\n'.join(questions)
template = (
            "You are an expert product manager. Based on the initial idea and any relevant search results, "
            "provide a comprehensive and insightful answer to the following questions. Each answer will be displayed to the client for helping them decide their answer. So make each answer as informative as possible with recommendations, options, industry standards, examples of existing products with simiiar focus etc.\n\n"
            "Initial Idea: {initial_idea}\n"
            "Questions: \n {questions}\n"
            "Include reasoning, multiple options where applicable, and industry best practices.\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"questions": str_questions, 
            "initial_idea": initial_idea,
            "format_instructions": format_instructions}
initial_qa = interviewer._run_llm_step(dict(), template, params, "Could not generate an initial answer due to an error.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)

2025-07-31 01:40:50,479 - INFO - Executing LLM chain with params: dict_keys(['questions', 'initial_idea', 'format_instructions'])
2025-07-31 01:41:10,330 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-31 01:41:10,347 - INFO - LLM chain executed successfully.


## Agent 2: Create Web search from the answers for enriching the Answers Further

In [10]:
class Websearch(BaseModel):
    search_queries: List[str]

parser = PydanticOutputParser(pydantic_object=Websearch)
format_instructions = parser.get_format_instructions()

template = (
            "You are an expert product manager. Based on the initial idea and the initial QA with the client, generate a list of search queries to perform web search, to enrich the answers"
            "Each answer will be displayed to the client for helping them decide their answer. So make the queries to extract relevant and timely information. Aim to enrich the recommendations, options, industry standards, examples of existing products with simiiar focus etc.\n\n"
            "Initial Idea: {initial_idea}\n"
            "QA: \n {initial_qa}\n"
            "Include reasoning, multiple options where applicable, and industry best practices.\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"initial_idea": initial_idea,
            "format_instructions": format_instructions,
            "initial_qa": initial_qa.str()}

websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)

2025-07-31 01:52:18,155 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions', 'initial_qa'])
2025-07-31 01:52:23,207 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-31 01:52:23,220 - INFO - LLM chain executed successfully.


## Information Gatherer Workflow

## Agent 3: Generating a List of Searches based on the user idea

In [24]:
class Websearch(BaseModel):
    search_queries: List[str]

parser = PydanticOutputParser(pydantic_object=Websearch)
format_instructions = parser.get_format_instructions()

template = (
            "You are a useful product research and gap analysis agent. Based on the initial product idea from the user below, you will generate a list of queries to research the product idea from scratch. "
            "Focus on the following: "
            "1. Exisitng solution similiar to the user idea"
            "2. Typical Feature set for a product with similiarity to the user idea"
            "3. Examples of typical User journey for a product with similiarity to the user idea"
            "4. Typical tech stack for a product with similiarity to the user idea"
            "5. Exisitng codebases in Github which can be leveraged for implementing the user idea"
            "6. Third party python packages or open source projects / packages that can be used for implementing the user idea"
            "7. Examples of UI / UX for a product with similiarity to the user idea"
            "8. Main functional modules for the product"
            "9. Gap analysis vs. existing solutions"
            "10. Features that are not required for the MVP"
            "11. Features that are useful but not neccessary"
            "12. Data Sources if relevant, API details"
            "Return a maximum of 30 queries, but feel free to return less if that is sufficient. Make sure each query is unique and not similar to each other: \n"
            "Initial Idea: {initial_idea}\n"
            "Answer Format Instructions: "
            "\n{format_instructions}"
        )

params = {"initial_idea": initial_idea,
        "format_instructions": format_instructions,
        }

websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)
print('\n'.join(websearch.search_queries))

2025-08-02 15:01:02,119 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions'])
2025-08-02 15:01:07,596 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-02 15:01:07,602 - INFO - LLM chain executed successfully.


Existing AI-powered mutual fund recommendation platforms in India
Popular mutual fund analysis tools for Indian investors
Feature comparison of Indian mutual fund tracking apps
User journey examples for mutual fund investment platforms
How do users interact with mutual fund recommendation systems?
Typical tech stack for financial analytics web applications
Open source mutual fund analysis projects on GitHub
Python packages for financial data analysis and visualization
APIs providing Indian mutual fund data (e.g., Value Research, Morningstar, AMFI)
Best practices for visualizing mutual fund performance data
Modules required for a mutual fund recommendation engine
Gap analysis: AI-powered mutual fund recommendation vs. existing Indian solutions
Features commonly found in mutual fund tracking apps
Features that can be excluded from an MVP mutual fund recommendation system
Useful but non-essential features for mutual fund analytics platforms
UI/UX design examples for investment recommendat

In [ ]:
# Use OPEN AI with WebBrowser tool to get the results

async def fetch_websearch_results(websearch):
    client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    tasks = []
    for query in websearch.search_queries:
        search_task =  client.responses.create(
            model="gpt-4.1",  # or another supported model
            input="Use web_search to search for information for each of the following topics - \n" + query,
            tools=[
                {
                "type": "web_search"
            }
            ]
        )
        tasks.append(search_task)

    results = await asyncio.gather(*tasks)
    search_results = [response.output[1].content[0].text for response in results]
    return search_results

search_results = await fetch_websearch_results(websearch)

2025-08-02 15:01:20,294 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:20,453 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:20,807 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:20,999 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,080 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,113 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,147 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,203 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,364 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:01:21,504 - INFO - HTTP Request:

In [ ]:
MAX_DEPTH = 5

depth = 1
current_learnings = []
STOP = False

while not STOP and depth <= MAX_DEPTH:
    logging.info("\nDepth: " + str(depth))
    
    for query, result in zip(websearch.search_queries, search_results):
        current_learnings.append("Topic: " + query + "\nResult: " + result)

    total_learnings = len(current_learnings)

    logging.info("Total learnings: " + str(total_learnings))
    logging.info("*******************************************")
    learning_text = "========\n".join(current_learnings)

    template = (
                "You are evaluating the work of a research agent who was tasked to extract information from the web about a product idea suggested by the user"
                "Initial Idea: {initial_idea}\n"
                "Information extracted by the research agent: \n {topics}\n"
                "Evaluate if you have sufficient information that the client might need to decide about how to execute the product idea. If not, generate a list of additional queries to extract information from the web."
                "If the information set is sufficient, return an empty list."
                "Answer Format Instructions: "
                "\n{format_instructions}"
            )

    params = {"initial_idea": initial_idea,
            "format_instructions": format_instructions,
            "topics": learning_text,
            }

    websearch = interviewer._run_llm_step(dict(), template, params, "Could not generate the list of web search queries.", output_parser=parser, model_override="gpt-4.1", temparature_override=0.1)
    if len(websearch.search_queries) == 0:
        logging.info("No additional queries generated.")
        STOP = True
    else:
        logging.info("No additional queries generated.")
        logging.info('\n'.join(websearch.search_queries))
        depth += 1
        search_results = await fetch_websearch_results(websearch)
    logging.info("************************************************")

2025-08-02 15:01:49,653 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions', 'topics'])



Depth: 1
Total learnings: 0
*******************************************


2025-08-02 15:01:52,878 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-02 15:01:52,886 - INFO - LLM chain executed successfully.


No additional queries generated.
List of public and third-party APIs for Indian mutual fund data
Regulatory requirements for building mutual fund advisory platforms in India
Best practices for AI-powered investment recommendation systems
Existing competitors in Indian mutual fund recommendation space
Available visualization libraries for financial data in India
How to track user investments in Indian mutual funds via APIs
Methods for risk profiling in mutual fund advisory
Optimal asset allocation algorithms for mutual funds
Switch suggestion logic for mutual fund portfolios
Performance analysis metrics for mutual funds in India
Integration options with Indian mutual fund platforms (e.g., CAMS, Karvy)
User authentication and data privacy requirements for financial apps in India


2025-08-02 15:02:01,653 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:02,371 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:02,821 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:02,878 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:02,923 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:03,546 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:04,305 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:04,576 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:04,740 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:04,896 - INFO - HTTP Request:

...

No additional queries generated.
What are the costs, licensing, and commercial terms for using each mutual fund data API in India?
Are there any open-source or free-to-use Indian mutual fund data APIs suitable for commercial applications?
What are the technical integration requirements (API limits, latency, data refresh rates) for leading Indian mutual fund APIs?
What are the most common user interface/UX patterns for mutual fund recommendation and tracking apps in India?
What are the challenges and best practices for onboarding users (KYC, e-signature, etc.) in Indian mutual fund platforms?
What are the most effective AI/ML models used by Indian mutual fund recommendation platforms, and are there any published benchmarks?
What are the typical timelines and costs for SEBI Investment Adviser registration and compliance setup for a new fintech?
Are there any sandbox or pilot programs by SEBI or RBI for fintechs building mutual fund advisory solutions?
What are the most common customer ac

2025-08-02 15:02:17,818 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:18,542 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:19,211 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:19,578 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:19,696 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:20,330 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:20,665 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:20,669 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:20,745 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-02 15:02:20,840 - INFO - HTTP Request:

************************************************

Depth: 3
Total learnings: 26
*******************************************


2025-08-02 15:02:30,712 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-02 15:02:30,746 - INFO - LLM chain executed successfully.


No additional queries generated.
************************************************


In [3]:

from ProductDesigner.agents.research_subgraph_agent import ResearchSubgraphAgent, ResearchState, _execute_parallel_searches_for_queries

agent = ResearchSubgraphAgent()
research_state = ResearchState(
                initial_idea=initial_idea,
                max_depth=5
            )

2025-08-05 10:29:49,330 - INFO - LLMService initialized with model: gpt-4o


In [4]:
research_state = agent._generate_queries_node(research_state)

2025-08-05 10:29:53,337 - INFO - Generating initial search queries...
2025-08-05 10:29:53,339 - INFO - Executing LLM chain with params: dict_keys(['initial_idea', 'format_instructions'])
2025-08-05 10:30:00,107 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-05 10:30:00,121 - INFO - LLM chain executed successfully.
2025-08-05 10:30:00,124 - INFO - Generated 30 search queries
2025-08-05 10:30:00,125 - INFO - 1. Existing AI-powered mutual fund recommendation platforms in India
2025-08-05 10:30:00,126 - INFO - 2. Popular mutual fund tracking apps in India
2025-08-05 10:30:00,126 - INFO - 3. Feature comparison of Indian mutual fund analysis tools
2025-08-05 10:30:00,127 - INFO - 4. User journey for mutual fund investment and tracking apps
2025-08-05 10:30:00,129 - INFO - 5. Tech stack used by leading Indian fintech apps for mutual funds
2025-08-05 10:30:00,129 - INFO - 6. Open source mutual fund analysis projects on Github
2025-08-05 10:30:0

In [ ]:
research_state =  agent._execute_parallel_search_node(research_state)

2025-08-05 10:32:26,434 - INFO - ---1 ---
2025-08-05 10:32:26,435 - INFO - --- Existing AI-powered mutual fund recommendation platforms in India ---
2025-08-05 10:32:26,436 - INFO - --- openai_web_search ---
2025-08-05 10:32:26,436 - INFO - --- firecrawl ---
2025-08-05 10:32:26,437 - INFO - --- Popular mutual fund tracking apps in India ---
2025-08-05 10:32:26,437 - INFO - --- openai_web_search ---
2025-08-05 10:32:26,438 - INFO - --- firecrawl ---
2025-08-05 10:32:26,439 - INFO - --- Feature comparison of Indian mutual fund analysis tools ---
2025-08-05 10:32:26,440 - INFO - --- openai_web_search ---
2025-08-05 10:32:26,440 - INFO - --- firecrawl ---
2025-08-05 10:32:26,442 - INFO - --- User journey for mutual fund investment and tracking apps ---
2025-08-05 10:32:26,442 - INFO - --- openai_web_search ---
2025-08-05 10:32:26,443 - INFO - --- firecrawl ---
2025-08-05 10:32:26,443 - INFO - --- Tech stack used by leading Indian fintech apps for mutual funds ---
2025-08-05 10:32:26,443 - 

# 2. Stage 1: Web Search for Context

In [12]:
# Create a search query and fetch context from the web
class SearchResuts(BaseModel):
    search_queries: List[str] = []
    search_results: List[str] = []

    def str(self):
        return '\n'.join([f"Query: {qa[0]}\n Search Results: {qa[1]}" for qa in zip(self.search_queries, self.search_results)])

search_results = SearchResuts()
for query in websearch.search_queries:
    search_result = interviewer.search_web_for_context(query)
    search_results.search_queries.append(query)
    search_results.search_results.append(search_result)

print("--- Web Search Results ---")
print(search_results)

2025-07-31 01:57:10,031 - INFO - Performing web search for: latest AI-powered personal finance apps 2024
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-07-31 01:57:10,529 - INFO - response: https://www.bing.com/search?q=latest+AI-powered+personal+finance+apps+2024&filters=ex1%3A%22ez5_19934_20299%22 200
2025-07-31 01:57:10,562 - INFO - Web search successful.
2025-07-31 01:57:10,565 - INFO - Performing web search for: best budgeting and investment management apps comparison
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-07-31 01:57:10,934 - INFO - response: https://www.bing.com/search?q=best+budgeting+and+investment+management+

--- Web Search Results ---
search_queries=['latest AI-powered personal finance apps 2024', 'best budgeting and investment management apps comparison', 'AI in personal finance: industry trends and standards', 'examples of conversational AI in finance apps', 'how Cleo, Mint, YNAB, and Personal Capital use AI', 'top financial data aggregation APIs 2024', 'security best practices for fintech applications', 'GDPR and CCPA compliance for personal finance apps', 'UI/UX design trends for finance and investment apps', 'case studies of AI-driven financial coaching apps', 'predictive analytics in personal finance applications', 'user engagement metrics for finance apps', 'successful onboarding flows in fintech apps', 'role-based access control in financial applications', 'mobile-first design for personal finance tools', 'open source libraries for financial data visualization', 'Plaid vs Yodlee vs Salt Edge: feature comparison', 'how to implement end-to-end encryption in fintech apps', 'OAuth2 aut

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
interviewer.search_tool = DuckDuckGoSearchResults()
query = websearch.search_queries[0]
list_results = interviewer.search_tool.invoke(query)

2025-07-31 02:21:21,458 - INFO - Performing web search for: latest AI-powered personal finance apps 2024
c:\Code\anaconda3\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
2025-07-31 02:21:21,723 - INFO - response: https://www.bing.com/search?q=latest+AI-powered+personal+finance+apps+2024&filters=ex1%3A%22ez5_19934_20299%22 200
2025-07-31 02:21:23,258 - INFO - response: https://www.bing.com/search?q=latest+AI-powered+personal+finance+apps+2024&filters=ex1%3A%22ez5_19934_20299%22&first=11&FORM=PERE 200
2025-07-31 02:21:23,370 - INFO - Web search successful.


In [22]:
list_results

'snippet: Nov 6, 2024 · Here are some of the most popular, freely available, AI-powered budgeting and financial management tools used in India. The table below compares these tools based on …, title: Top 5 AI Budgeting Tools - Analytics Vidhya, link: https://www.analyticsvidhya.com/blog/2024/11/ai-budgeting-tools/, snippet: Jun 29, 2025 · AI finance tools are mobile apps or platforms that use artificial intelligence to track, analyze, and manage your personal finances. These tools study your spending habits, give …, title: Top AI Tools for Personal Finance in India: Your Smart Money …, link: https://taxreaders.com/top-ai-tools-for-personal-finance-in-india-your-smart-money-manager-in-2025-ai-tools-for-personal-finance-in-india/, snippet: Sep 21, 2024 · Managing personal finances can often feel overwhelming. Thankfully, AI-powered tools have emerged to simplify budgeting, saving, and managing expenses., title: 10 OF BEST & COMMONLY USED AI TOOLS FOR PERSONAL …, link: https://aienabledhu

# 3. Stage 2: Generate Initial Answer

In [ ]:
# Get the first question and generate an initial answer based on the search context
question = interviewer.get_next_question()
initial_answer = interviewer.generate_initial_answer(mock_state, question, search_results)

print(f"--- Initial Answer for: '{question}' ---")
print(initial_answer)

# 4. Stage 3: Critique Answer

In [ ]:
# Critique the generated answer for weaknesses or omissions
critique = interviewer.critique_answer(mock_state, question, initial_answer)

print("--- Critique of the Answer ---")
print(critique)

# 5. Stage 4: Refine Answer

In [ ]:
# Refine the initial answer based on the critique
refined_answer = interviewer.refine_answer(mock_state, question, initial_answer, critique)

print("--- Refined Answer ---")
print(refined_answer)

# 6. Stage 5: Format for User Presentation

In [ ]:
# Format the refined answer into a user-friendly presentation
user_presentation = interviewer.present_to_user(mock_state, question, refined_answer)

print("--- Formatted for User ---")
print(user_presentation)

# 7. Full Self-Critique Workflow (All Stages Combined)

In [ ]:
# Run the entire self-critique process for a single question
print(f"Running full self-critique for question: '{question}'")
full_result = interviewer.process_question_with_self_critique(mock_state, question)

if 'error' in full_result:
    print(f"An error occurred: {full_result['error']}")
else:
    print("--- Final User Presentation from Full Workflow ---")
    print(full_result.get('user_presentation'))

# 8. Full Interview Simulation

In [ ]:
# Simulate running the agent through the first 3 questions of the interview
agent_for_run = SafeProductInterviewerAgent()
current_state = {"initial_idea": initial_idea, "qna_history": {}}

for i in range(3):
    print(f'--- Running Interview: Question {i+1} ---')
    current_state = agent_for_run.run(current_state)
    print(f'Q&A history now contains {len(current_state["qna_history"])} items.')
    print('--- End of Question ---
')

print("Full interview simulation complete.")